# A Data Analysis on Just the Rides CSV File

Check the following questions:
1. incompleted rides
    -> requested, but canceled
    -> accepted, but canceled
    -> arrived at, but canceled
    -> picked up at, but canceled
    
2. completed rides
    -> all 5 from same ride id, -> mark rides completed, or not column

3. Time of ride (day) - earlymorning, morning, afternoon, evening, late-evening
    -> 
    
4. Time of ride (week)  - weekday, weekend, friday night?
    ->

3. Category type of ride - timely, versus long-distance
    -> ratio of distance to cost vs duration to the cost
##### A2. Ride category - time vs distance

Done on other df

In [1]:
# Necessary libs
import os
import numpy as np
import pandas as pd
import scipy
import sklearn
import datetime


# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dabest

# custom libs
import sys
sys.path.append("../")
from src.data.rider import Fare, Ride

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2

# 1. Load in Data

In [2]:
# datadir = "../data/interim/"
# fpath = os.path.join(datadir, "driver_v1.csv")
                              
# df = pd.read_csv(fpath)

# print(os.listdir(datadir))

In [3]:
datadir = "../data/raw/"

ridetime_fpath = os.path.join(datadir, "ride_timestamps.csv")
                              
time_df = pd.read_csv(ridetime_fpath)

print(os.listdir(datadir))

['.gitkeep', 'ride_timestamps.csv', 'driver_ids.csv', 'ride_ids.csv']


In [4]:
display(time_df.head())

print("The size of the dataframes are: ", time_df.shape)

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


The size of the dataframes are:  (970405, 3)


In [5]:
# ensure datetime
time_df['timestamp'] = pd.to_datetime(time_df['timestamp'])

display(time_df.head())

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


##### A1. Incomplete/Complete ride

In [6]:
check_df = time_df.copy()
check_df.set_index("ride_id", drop=True, inplace=True)
check_df.drop(labels=["timestamp"], inplace=True, axis=1)
check_dict = check_df.to_dict('list')

display(check_df.head())
print(len(check_dict))

,event
ride_id,
00003037a262d9ee40e61b5c0718f7f0,requested_at
00003037a262d9ee40e61b5c0718f7f0,accepted_at
00003037a262d9ee40e61b5c0718f7f0,arrived_at
00003037a262d9ee40e61b5c0718f7f0,picked_up_at
00003037a262d9ee40e61b5c0718f7f0,dropped_off_at


1


In [7]:
# check each ride_id as "incompleted", or "completed"
complete_event = [
    "requested_at",
    "accepted_at",
    "arrived_at",
    "picked_up_at",
    "dropped_off_at",
]

complete_rides = {}
num_incomplete_rides = 0

for i, (_id, events) in enumerate(check_dict.items()):
    if not all(x in events for x in complete_event):
        complete_rides[_id] = 0
        num_incomplete_rides += 1
    else:
        complete_rides[_id] = 1

print("Number incomplete rides: ", num_incomplete_rides)
print("Total number of rides: ", len(time_df))

Number incomplete rides:  0
Total number of rides:  970405


In [8]:
display(time_df.head())

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


##### A2. Ride time of day

In [9]:
def compute_hr_day(df):
    x = df["timestamp"]
    hour=x.dt.time
    return hour

time_df["hr_of_day"] = time_df.pipe(compute_hr_day)
display(time_df.head())

,ride_id,event,timestamp,hr_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05


In [10]:
late_evening = ["22:00:00", "02:00:00"]
early_morning = ["02:00:00", "06:00:00"]
late_morning = ["06:00:00", "10:00:00"]
early_after = ["10:00:00", "14:00:00"]
late_after = ["14:00:00", "18:00:00"]
early_evening = ["18:00:00", "22:00:00"]

ranges = [early_morning, late_morning, early_after, late_after, early_evening, late_evening]
for i, tperiod in enumerate(ranges):
    ranges[i] = [datetime.datetime.strptime(x, '%H:%M:%S') for x in tperiod]
print(ranges)

[[datetime.datetime(1900, 1, 1, 2, 0), datetime.datetime(1900, 1, 1, 6, 0)], [datetime.datetime(1900, 1, 1, 6, 0), datetime.datetime(1900, 1, 1, 10, 0)], [datetime.datetime(1900, 1, 1, 10, 0), datetime.datetime(1900, 1, 1, 14, 0)], [datetime.datetime(1900, 1, 1, 14, 0), datetime.datetime(1900, 1, 1, 18, 0)], [datetime.datetime(1900, 1, 1, 18, 0), datetime.datetime(1900, 1, 1, 22, 0)], [datetime.datetime(1900, 1, 1, 22, 0), datetime.datetime(1900, 1, 1, 2, 0)]]


In [11]:
display(time_df.head())

,ride_id,event,timestamp,hr_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05


In [12]:
# drop nan rows
print(time_df.shape)
time_df=time_df.dropna()
print(time_df.shape)

(970405, 4)
(970404, 4)


In [13]:
def bin_f(x):
    if x.time() < datetime.time(6):
#         return "00:00-05:59"
        return 1
    elif x.time() < datetime.time(12):
#         return "06:00-11:59"
        return 2
    elif x.time() < datetime.time(18):
#         return "12:00-17:59"
        return 3
    else:
#         return "18:00-23:59"
        return 4

time_df["time_of_day"] = time_df["timestamp"].apply(bin_f)

display(time_df.head())

,ride_id,event,timestamp,hr_of_day,time_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:39:19,2
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:39:51,2
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:44:31,2
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:44:33,2
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,10:03:05,2


In [14]:
# only keep the time of day wrt to categorization
time_df.drop("hr_of_day", inplace=True, axis=1)
display(time_df.head())

,ride_id,event,timestamp,time_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2


##### A3. Ride time of week

In [15]:
time_df['weekday'] = ((pd.DatetimeIndex(time_df.timestamp).dayofweek) // 5 == 1).astype(float)
display(time_df.head())

,ride_id,event,timestamp,time_of_day,weekday
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2,0.0
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2,0.0
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2,0.0
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2,0.0
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2,0.0


##### A4. Ride time of year

In [16]:
def compute_month(df):
    x = df["timestamp"]
    month=x.dt.month
    return month

time_df["month"] = time_df.pipe(compute_month)
display(time_df.head())

,ride_id,event,timestamp,time_of_day,weekday,month
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,2,0.0,6
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,2,0.0,6
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,2,0.0,6
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,2,0.0,6
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,2,0.0,6


In [17]:
mindate = time_df["timestamp"].min()
maxdate = time_df["timestamp"].max()

print("The range of all rides for our data is: ", mindate, maxdate)

The range of all rides for our data is:  2016-03-28 05:48:18 2016-06-27 00:50:50


# Compute week index - For churn computation based on rides

In [33]:
week_start_dates = ['2016-03-28',
                    '2016-04-04',
                    '2016-04-11',
                    '2016-04-18',
                    '2016-04-25',
                    '2016-05-02',
                    '2016-05-09',
                    '2016-05-16',
                    '2016-05-23',
                    '2016-05-30',
                    '2016-06-06',
                    '2016-06-13',
                    '2016-06-20',
                    '2016-06-27'
        ]

def categorize_ride_by_week(df):
    weekindex = (df["date"] - mindate).dt.days
    weekindex = weekindex // 7 + 1
    return weekindex
    
mindate = time_df["timestamp"].min().date()
print(mindate)

time_df["date"] = time_df["timestamp"].dt.date
time_df["week_index"] = time_df.pipe(categorize_ride_by_week)
time_df[time_df["week_index"]==14] = 13
display(time_df.head())

2016-03-28


,ride_id,event,timestamp,time_of_day,weekday,month,date,week_index
35061,092ec855c70d70df6d4be5fb23e29081,accepted_at,2016-03-29 11:26:43,2,0.0,3,2016-03-29,1
619201,a36318bb575fd2e0adefe8a9d5ecf5b3,accepted_at,2016-03-31 10:53:00,2,0.0,3,2016-03-31,1
619202,a36318bb575fd2e0adefe8a9d5ecf5b3,arrived_at,2016-03-31 10:57:20,2,0.0,3,2016-03-31,1
619203,a36318bb575fd2e0adefe8a9d5ecf5b3,picked_up_at,2016-03-31 10:57:23,2,0.0,3,2016-03-31,1
619204,a36318bb575fd2e0adefe8a9d5ecf5b3,dropped_off_at,2016-03-31 11:23:02,2,0.0,3,2016-03-31,1


In [32]:

time_df.sort_values(by="week_index", inplace=True)
display(time_df.head(3))
display(time_df.tail())

,ride_id,event,timestamp,time_of_day,weekday,month,date,week_index
35061,092ec855c70d70df6d4be5fb23e29081,accepted_at,2016-03-29 11:26:43,2,0.0,3,2016-03-29,1
619201,a36318bb575fd2e0adefe8a9d5ecf5b3,accepted_at,2016-03-31 10:53:00,2,0.0,3,2016-03-31,1
619202,a36318bb575fd2e0adefe8a9d5ecf5b3,arrived_at,2016-03-31 10:57:20,2,0.0,3,2016-03-31,1


,ride_id,event,timestamp,time_of_day,weekday,month,date,week_index
724442,bf49073d4b4369bb0e08a2d36e1cd097,arrived_at,2016-06-27 00:01:06,1,0.0,6,2016-06-27,14
769039,cb283287700d34c276682909c36aa99f,dropped_off_at,2016-06-27 00:01:35,1,0.0,6,2016-06-27,14
185164,311542541524c0d356fa8d7fc229baae,dropped_off_at,2016-06-27 00:05:30,1,0.0,6,2016-06-27,14
944239,f90966687aebca6cc9efcd9e42897b75,dropped_off_at,2016-06-27 00:26:35,1,0.0,6,2016-06-27,14
944237,f90966687aebca6cc9efcd9e42897b75,arrived_at,2016-06-27 00:02:03,1,0.0,6,2016-06-27,14


# Save New Dataset on Rides

In [36]:
fpath = "../data/interim/ride_timestamps_v2.csv"

In [37]:
time_df.to_csv(fpath, index=False)

# Random Exploration

In [34]:
test = time_df.groupby(pd.Grouper(key="week_index"))

In [35]:
display(test.head())

,ride_id,event,timestamp,time_of_day,weekday,month,date,week_index
35061,092ec855c70d70df6d4be5fb23e29081,accepted_at,2016-03-29 11:26:43,2,0.0,3,2016-03-29,1
619201,a36318bb575fd2e0adefe8a9d5ecf5b3,accepted_at,2016-03-31 10:53:00,2,0.0,3,2016-03-31,1
619202,a36318bb575fd2e0adefe8a9d5ecf5b3,arrived_at,2016-03-31 10:57:20,2,0.0,3,2016-03-31,1
619203,a36318bb575fd2e0adefe8a9d5ecf5b3,picked_up_at,2016-03-31 10:57:23,2,0.0,3,2016-03-31,1
619204,a36318bb575fd2e0adefe8a9d5ecf5b3,dropped_off_at,2016-03-31 11:23:02,2,0.0,3,2016-03-31,1
...,...,...,...,...,...,...,...,...
410368,6cbdcb40857863f10befbf81d5e34bae,picked_up_at,2016-06-22 12:50:51,3,0.0,6,2016-06-22,13
350036,5c615f884ac9d05f70d77c37a79eb5ad,accepted_at,2016-06-23 00:57:00,1,0.0,6,2016-06-23,13
89563,17d4e94008ec3e4d52547862de6ddc60,picked_up_at,2016-06-26 01:43:45,1,1.0,6,2016-06-26,13
350032,5c6155cd6cbbaa47af64a5d3c637e5fd,arrived_at,2016-06-23 14:54:52,3,0.0,6,2016-06-23,13
